### 4.1 Recurrent Neural Network适用场景

#### 一. 适用场景  
<img src='../img/sequencemodelexample1.png' height='70%' width='70%'>

#### 二. 符号表示  
一个识别人名的例子. $x$ : 输入一句话(单词序列), $y$ : 输出句子中该位置的单词是否为人名(0/1序列)
<img src='../img/motivaternn.png' height='70%' width='70%'>

### 4.2 Recurrent Neural Network模型

#### 一. 为什么使用Recurrent Neural Network
1. 普通神经网络   
 上述识别人名的例子, 使用普通神经网络训练的结果并不好, 因为它把每个单词作为不同的输入特征, 彼此独立的输入到神经网络. 没有表示出输入的单词之间的语序关系

2. 循环神经网络架构  
  1. 循环神经网络从左到右依次读入单词, 每个单词被不同时间步的单元处理
  2. 每个时间步的RNN单元之间参数共享.   
   如每个时间步中, 由${ x }^{ <t> }$到${ a }^{ <t> }$的权值矩阵$W_{ax}$相同  
   由${ a }^{ <t-1> }$到${ a }^{ <t> }$的权值矩阵$W_{aa}$相同  
   由${ a }^{ <t> }$到${ y }^{ <t> }$的权值矩阵$W_{ya}$相同  
  3. 如下图所示, RNN网络有两种表示方式, 左侧未展开的RNN网络. 右侧为循环形式的表述
  <img src='../img/rnnmodel1.png' width='100%' height='100%'>
  
#### 二. Forward Propagation
1. 正向传播计算  
  1. 每个时间步的输出$y^{<t>}$由当前时间步学习到的内容$a^{<t>}$决定
  2. 每个时间步学习到的内容$a^{<t>}$, 由上一个时间步学习到的内容$a^{<t-1>}$和本时间步的输入$x^{<t>}$共同决定
  3. 参数$W_{aa},W_{ax},W_{ya}$在时间步上共享  
  4. 输出$y$的激活函数, 和输出$a$的激活函数往往不是一个函数.  
   输出$y$的激活函数, 根据需要输出的情况不同而选择. 比如:二分类问题选择$sigmoid$, 多分类问题选择$softmax$  
   输出$a$的激活函数, 通常选择$tanh$或$Relu$
<img src='../img/rnnforward1.png' width='80%' height='80%'>
2. 公式的简化表达  
 在计算每个时间步学习到的信息时, 公式${ a }^{ <t> }=g\left( { W }_{ aa }{ a }^{ <t-1> }+{ W }_{ ax }{ x }^{ <t> }+{ b }_{ a } \right) $, 可以合并为${ a }^{ <t> }=g\left( { W }_{ a }({ a }^{ <t-1> },{ x }^{ <t> })+{ b }_{ a } \right) $
<img src='../img/briefrnn.png' width='40%' height='40%'>

#### 三. 反向传播
1. RNN的损失函数   
 RNN的每个时间步有一个损失函数. 为该时间步输出的交叉熵(标准逻辑回归的损失函数)  
 ${ L }^{ <t> }({ y }^{ <t> },{ \hat { y }  }^{ <t> })=-{ y }^{ <t> }\log { { \hat { y }  }^{ <t> } } -(1-{ y }^{ <t> })\log { (1-{ \hat { y }  }^{ <t> }) } $
2. Recurrent Network反向传播有两条路径  
  1. 垂直方向 : 每个时间步的输出$y^{<t>}$在本时间步内的反向传播, 更新$W_{y}$. 
  2. 水平方向 : 上个时间步学习到的信息在本时间步内的权重$W_{ax}$. 该跨时间步的反向传播也是根据$L^{<t+1>}$梯度下降而来
<img src='../img/rnnbackword1.png' height=60% width=60% >
3. 如图所示, RNN的反向传播要从最右面$L^{<T>}$开始计算. 才能保证横纵两个方向都能梯度下降

#### 四. RNN的梯度小时与梯度爆炸
RNN由于时间步之间会传递学习到的知识, 所以极易在跨越多个时间步传递时, 发生梯度爆炸和梯度消失
1. 梯度爆炸 
 Grident clipping : 发生梯度爆炸时, 会导致计算溢出出现NaN值. 可以设置一个阈值, 当梯度超过这个阈值后, 重新缩放梯度到一个小范围
2. 梯度消失  
 梯度爆炸会导致神经网络参数混乱, 效果大大降低, 但解决办法简单, 梯度clipping即可.  
 梯度消失会导致前面时间步的消息无法传递到后面的时间步, 这要从网络结构上加以解决(GRU/LSTM)

### 4.4 $T_X\neq T_Y$的RNN网络

#### 一.  $T_X\neq T_Y$的RNN网络
1. `many2one`  
  1. 情感分析中, 常常输入一个句子, 输出$0/1$, 表示喜欢或不喜欢. 因此这是一个`many2one`的网络  
  2. `many2one`的RNN, 每个时间步有一个输入$x^{<t>}$, 但只在最后一个时间步有输出 $ \hat { y }$
2. `one2many`  
  1. 音乐生成 : 输入一个数字, 表示音乐风格, 产生一段音符序列
  2.  `one2many` 网络, 输出$y^{<t-1>}$的单元, 会把学习到的知识$a^{<t-1>}$送给下一个时间步的RNN单元
  <img src="../img/many2onw&one2many.png">
3. $T_X\neq T_Y$的`many2many`  
  1. 机器翻译 : 输入法文序列, 输出英文序列. 这两个序列的长度可能不同
  2. 由于$T_X\neq T_Y$, 因此处理 $x$ 与处理 $y$ 的RNN单元是分开的
  <img src="../img/many2many.png">

### 4.5 RNN语言模型

#### 一. 何为语言模型
1. 何时使用语言模型:   
  如在语音识别领域, 语音可能被识别成发音相似的2个语句文本, 那么到底选择哪个文本输出呢, 需要根据这两个语句生成的概率比较决定
2. 语言模型的任务  
 语言模型任务为 : 输入一个语句. 该语句作为RNN网络的$y^{<1>}$,$y^{<2>}$..$y^{<T_{y}>}$进行训练. 最后输出这个语句的概率
 
#### 二. 语言模型的例子
<img src='../img/rnnlangmodel.png'>


#### 三. 使用语言模型生成小说
1. 以上, 已经可以根据语料库中的语句学习出RNN语言模型. 若语料库中的文章出自同一个作者, 则该模型就学习到了该作者的文风, 可以通过该模型重新采样生成一篇新文章, 其风格与该作者相同
2. 模型生成新文章的步骤  
 模型训练完毕以后, RNN中从输入到输出softmax的参数已定. 因此只要重新输入$x^{<0>}$就能持续产生文章
  1. 输入$x^{<0>}=\overrightarrow { 0 } $, 第一个时间步输出${ \hat { y }  }^{ <1> }$. 然后输入${ x }^{ <2> }={ \hat { y }  }^{ <1> }$, 得出第二个时间步的输出${ \hat { y }  }^{ <2> }$. 以此类推, 依次把上一时间步的输出作为下一时间步的输入, 从而产生小说序列
  2. 标点问题 : 若要产生的文档包含标点, 则用于训练模型的语料库中的文档要有标点, 且单词表中上需要有标点的位置
  3. 何时停止 :   
    1. 可以设置只生成20个单词的句子  
    2. 语料库中的文本集包含EOS标识, 让模型学习EOS的知识. 当生成文本中出现EOS标识时停止
<img src='../img/generatenoveal1.png' height='70%' width='70%'>    
3. 字符级和单词级的语言模型  
 一般训练RNN语言模型使用单次作为输入描述文本集. 有时想在生成的文档中区分大小写, 产生空格, 包含很多标点, 此时应使用字符描述每个文本, 同时语言模型每个时间步的输入是这些字符

### 4.6 Gated Recurrent Unit

#### 一. GRU新增组件
1. 用${ c }^{ <t> }$代替${ a }^{ <t> }$, 表示传递t时间步的记忆信息
2. ${ \widetilde { c }  }^{ <t> }$ : ${ c }^{ <t> }$在GRU单元内部计算的替代值
3. ${ \Gamma  }_{ u }$ :  当前时间步内计算的${ \widetilde { c }  }^{ <t> }$, 有多少权重用于更新记忆信息${ c }^{ <t> }$

#### 二. GRU单元图示
<img src='../img/gru1.png'>

#### 三. 为什么GRU能处理梯度消失问题
1. 普通RNN单元在传递长时间步的信息时, 梯度消失导致消息无法传递. 而GRU加入了更新门${ \Gamma  }_{ u }$解决这个问题
1. ${ \Gamma  }_{ u }$很容易=0, 只要W全职为很大的复数. ${ \Gamma  }_{ u }=0$导致${ c }^{ <t> }={ c }^{ <t-1> }$, 使得很早之前时间步的内容保留到很靠后的时间步 

### LSTM - Long short term memory
#### 一. LSTM单元架构
1. **LSTM中 $c^{<t>}\neq a^{<t>}$**  
 $c^{<t>}$ 表示t时间步的记忆信息, $a^{<t>}$ 表示t时间步学习到的信息
2. **门信息 : **   
 ${ a }^{ <t-1> }$与${ x }^{ <t> }$产生3个状态门 : 遗忘门,更新门, 输出门   
 ${ \Gamma  }_{ f/u/o }=\sigma \left( { W }_{ f/u/o }\left( { a }^{ <t-1> },{ x }^{ <t> } \right) +{ b }_{ f/u/o } \right) $
3. **本单元学习内容**    
 ${ \widetilde { c }  }^{ <t> }=tanh\left( { W }_{ c }\left( { a }^{ <t-1> },{ x }^{ <t> } \right) +{ b }_{ c } \right) $
4. **加权后的本单元记忆内容 = 更新门\*本单元学习内容+遗忘门\*上一单元记忆内容 **  
 ${ c }^{ <t> }={ \Gamma  }_{ u }*{ \widetilde { c }  }^{ <t> }+{ \Gamma  }_{ f }*{ c }^{ <t-1> }$
5. **本单元用于输出的中间内容=输出门\*tanh(本单元学习内容)**$a^{<t>}$    
 $a^{<t>}={ \Gamma  }_{ o }*tanh({ \widetilde { c }  }^{ <t> })$
 
#### 二. LSTM单元及LSTM网络示意图
<img src='../img/lstm.png'>

### 4.7 其他类型的RNN网络

#### 一. BRNN - 双向RNN
1. 双向RNN的RNN单元由一对GRU单元/一对LSTM单元组成. 一个BRNN单元包括一个前向传播的RNN单元和一个后向传播的RNN单元
2. 每个时间步的输出, 要同时依靠前向和后向的中间结果$\overrightarrow { a }$与$\overleftarrow { a } $
<img src="../img/brnn.png" width="58%" height="58%">

#### 二. DRNN - 深度循环网络
<img src="../img/drnn1.png">